<a href="https://colab.research.google.com/github/esusrutha/zeroshot/blob/main/file1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

# Read in the CSV files
RESULTS_DIR = 'drive/MyDrive'
df1 = pd.read_csv(RESULTS_DIR + '/D11.csv', encoding='latin1', engine='python')
df2 = pd.read_csv(RESULTS_DIR + '/D2.csv', encoding='latin1', engine='python')


# Merge the dataframes
merged_df = pd.concat([df1, df2], axis=0)

# Create a new column called 'category' and initialize it with 'None'
merged_df['category'] = None

# Create a list of adive,china,mask,news,transporation,usa and vaccine
Advice = ["['Stay at home']","['wash hands']","['wear mask']","['social distancing']"]
China = ["['Wuhan']","['China Coronavirus Updates']","['China news']","['other tweets related to China']"]
Mask = ["['Mask shortage']","['wear mask']","['mask types']","['N50']","['N95']","['3M8210']","['3M9001']","['3M9322']","['3M9501']"]
News = ["['Coronavirus updates']","['news']","['rules']"]
Transportation = ["['Flights']","['traffic']","['traveling']"]
USA = ["['U.S. Coronavirus Updates']","['COVID19']","['U.S. news']","['United States']","['US']","['USA']"]
Vaccine = ["['Vaccine news']","['vaccine progress']","['vaccine injection']"]

# Iterate over the rows of the dataframe
for index, row in merged_df.iterrows():
    # Check if the hashtags column contains  'Stay at home','wash hands','wear mask' or 'social distancing'
    if row['hashtags'] in Advice:
        # If it does, set the category to 'Advice'
        merged_df.at[index, 'category'] = 'Advice'
    elif row['hashtags'] in China:
        # If it does, set the category to 'China'
        merged_df.at[index, 'category'] = 'China'
    elif row['hashtags'] in Mask:
        # If it does, set the category to 'Mask'
        merged_df.at[index, 'category'] = 'Mask'
    elif row['hashtags'] in News:
        # If it does, set the category to 'News'
        merged_df.at[index, 'category'] = 'News'
    elif row['hashtags'] in Transportation:
        # If it does, set the category to 'Transportation'
        merged_df.at[index, 'category'] = 'Transportation'
    elif row['hashtags'] in USA:
        # If it does, set the category to 'USA'
        merged_df.at[index, 'category'] = 'USA'
    elif row['hashtags'] in Vaccine:
        # If it does, set the category to 'Vaccine'
        merged_df.at[index, 'category'] = 'Vaccine'
    
# Print the updated dataframe
print(merged_df["category"].value_counts())

Mask              469
Advice            311
USA               224
News              216
Vaccine           134
Transportation    132
China             131
Name: category, dtype: int64


In [4]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f2b3fe1d5b8fc1223feda3c0383648eecff899622f9ceeec76c2de0bda2835ec
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [4]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Read the merged DataFrame containing the tweets and categories
merged_df = merged_df = pd.concat([df1, df2], axis=0)

# Load the pre-trained S-BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Convert the tweets into sentence embeddings
tweet_embeddings = model.encode(merged_df['text'].values)

# Define the possible label names
label_names = ['Vaccine','USA','Transportation','News','Mask','China','Advice']

# Convert the label names into label embeddings
label_embeddings = model.encode(label_names)

# Calculate the cosine similarities between each tweet embedding and each label embedding
cos_similarities = cosine_similarity(tweet_embeddings, label_embeddings)

# Assign the label with the highest cosine similarity to each tweet
predicted_labels = [label_names[i] for i in cos_similarities.argmax(axis=1)]

# Add the predicted labels to the merged DataFrame
merged_df['predicted_label'] = predicted_labels

# Print the resulting DataFrame
print(merged_df)

KeyboardInterrupt: ignored